# Assignment 2
## Ruby Prajapati (231110042)
## Anshul Purohit (231110007)

# Importing modules

In [1]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import vtk
from vtk.util import numpy_support
import ipywidgets as widgets
from IPython.display import display
from skimage import measure
import matplotlib.cm as cm
from IPython.display import clear_output

# Loading Data

In [2]:
# Step 1: Load the 3D dataset
# Load the VTI dataset
vti_file = "mixture.vti"
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(vti_file)
reader.Update()

# Get the loaded grid
vtk_data = reader.GetOutput()


In [3]:
# Sample function to create isosurface plot
data_array = vtk_data.GetPointData().GetScalars()

# Convert the data array to a numpy array
numpy_data = np.array(data_array)


# Extracting grid information

In [4]:
# Extract grid information
dimensions = vtk_data.GetDimensions()
spacing = vtk_data.GetSpacing()
origin = vtk_data.GetOrigin()
dims = vtk_data.GetDimensions()

# Generate coordinates
x = np.linspace(origin[0], origin[0] + (dimensions[0] - 1) * spacing[0], dimensions[0])
y = np.linspace(origin[1], origin[1] + (dimensions[1] - 1) * spacing[1], dimensions[1])
z = np.linspace(origin[2], origin[2] + (dimensions[2] - 1) * spacing[2], dimensions[2])

# Create meshgrid
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# values 
values = numpy_data.reshape(dims, order='F')

# values = values[:100]
# X = X[:100]
# Y = Y[:100]
# Z = Z[:100]


In [5]:
# print(values[:10])
d = values.flatten()
print(d[:10])

[-0.04087147 -0.02041778 -0.0047331  -0.00140164 -0.0117143  -0.03537603
 -0.06927402 -0.10650624 -0.13915275 -0.16373235]


# Creating isosurface

In [6]:
def create_isosurface(data, isovalue):
    contour_value = isovalue # Example contour value

    # Create isosurface
    isosurface = go.Isosurface(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=values.flatten(),
        isomin=contour_value,
        isomax=contour_value,
        colorscale='plasma',
        cmin = values.min(),
        cmax = values.max(),
    )
    return isosurface

In [7]:
fig_hist = go.FigureWidget()

def update_histogram(iso_value):
    hist_data = values.flatten()
    if iso_value == 0:
    #         set the entire range as given in question
        scaler_data_range = hist_data 
    else:
        scaler_data_range = hist_data[((iso_value - 0.25) <= hist_data) & (hist_data <= (iso_value + 0.25))]
    histogram = go.Histogram(x=scaler_data_range, nbinsx=30)
    
    return histogram



# Creating reset button

In [8]:

# creating reset button 

reset_button = widgets.Button(description="Reset")

def reset_button_clicked(b):
    iso_slider.value = 0.0

reset_button.on_click(reset_button_clicked)

# Creating slider for isovalue

In [9]:
# Step 3: Create slider for isovalue
# iso_slider = widgets.FloatSlider(value=0.0, min=0.0, max=np.max(array), step=0.01, description='Isovalue')

step = 0.01
initial_value = 0.0

min_scalar_value = -0.99355406
max_scalar_value = 0.43280163

# Create the FloatSlider widget
iso_slider = widgets.FloatSlider(
    min=min_scalar_value,
    max=max_scalar_value,
    step=step,
    value=initial_value,
    description='Isoval:',
    layout={'width': '50%'},
    continuous_update=False
)


# Updating plots when slider values will change

In [10]:
def on_slider_change(change):
#     print('hi')
    clear_output(wait=True)
#     create_isosurface(numpy_data,change.new)
    
    isosurface_plot = go.FigureWidget(create_isosurface(numpy_data, change.new))
    # Update layout to adjust scene dimensions
    isosurface_plot.update_layout(
        scene=dict(
            aspectratio=dict(x=1, y=1, z=1),  # Set aspect ratio to make sure the plot is proportional
            xaxis=dict(showticklabels=False),  # Hide tick labels on x-axis
            yaxis=dict(showticklabels=False),  # Hide tick labels on y-axis
            zaxis=dict(showticklabels=False),  # Hide tick labels on z-axis
            camera=dict(eye=dict(x=1.3, y=1.3, z=1.3))  # Adjust camera position to view entire plot
        )
    )

    # Set the size of the plot
    isosurface_plot.update_layout(
        width=500,  # Set the width of the plot
        height=500  # Set the height of the plot
    )

    # Get the surface trace
    surface_trace = isosurface_plot.data[0]

    # Update showscale property to False
    surface_trace.showscale = False
    
    fig_hist = go.FigureWidget(update_histogram(change.new))
    fig_hist.update_layout(
        xaxis_title="Vortex scalar values",
        yaxis_title="Frequency",
        title="Histogram"
    )

    top = widgets.HBox([iso_slider,reset_button])
    bottom = widgets.HBox([isosurface_plot,fig_hist])
    display(widgets.VBox([top,bottom]))
    

iso_slider.observe(on_slider_change, names='value')

isosurface_plot = go.FigureWidget(create_isosurface(numpy_data, 0.0))

# Update layout to adjust scene dimensions
isosurface_plot.update_layout(
    scene=dict(
        aspectratio=dict(x=1, y=1, z=1),  # Set aspect ratio to make sure the plot is proportional
        xaxis=dict(showticklabels=False),  # Hide tick labels on x-axis
        yaxis=dict(showticklabels=False),  # Hide tick labels on y-axis
        zaxis=dict(showticklabels=False),  # Hide tick labels on z-axis
        camera=dict(eye=dict(x=1.3, y=1.3, z=1.3)),
#                    center=dict(x=-0.7, y=-0.5, z=-0.4))  # Adjust camera position to view entire plot
    )
)

# Set the size of the plot
isosurface_plot.update_layout(
    width=500,  # Set the width of the plot
    height=500  # Set the height of the plot
)

# Get the surface trace
surface_trace = isosurface_plot.data[0]

# Update showscale property to False
surface_trace.showscale = False

fig_hist = go.FigureWidget(update_histogram(0.0))
fig_hist.update_layout(xaxis_title="Vortex scalar values", yaxis_title="Frequency", title="Histogram")

top = widgets.HBox([iso_slider,reset_button])
bottom = widgets.HBox([isosurface_plot,fig_hist])
display(widgets.VBox([top,bottom]))

